In [1]:
import pandas as pd
import numpy as np

### Predict week 12
Need: 
- game stats from week 11
    - home team
    - away team
- prev game averages, last 5 games
    - home team
    - away team
- tweet stats leading up to week 12
    - home team
    - away team
- 

game_stats/wk_12_sentstats.csv 
- tweet stats for home and away
- score, spread 
- week, teams

final_stats/final_season_team_stats.csv
- prev game stats 
Game Stats for week 12 (week 11) = game_stats/final_season_team_stats.csv
- each row is individual team and week so would look up
- set index to team then week

In [71]:
cols_to_remove = [
    'ints_off',
    'pass_yds_per_att',
    'pass_net_yds_per_att',
    'pass_cmp_prcnt',
    'rush_yds_per_att',
    'fg_made',
    'fg_att',
    '4d_made',
    '4d_att',
    'exp_off',
    'exp_st',
    'off_pass_att',
    'off_pass_tds',
    'off_pass_ints'
]
def drop_redundant_cols(df):
    df = df.copy()
    df = df.drop(cols_to_remove, axis=1)
    return df

def time_to_float(x):
    splits = x.split(':')
    splits
    return int(splits[0]) + int(splits[1])/60
    

def time_of_poss_to_float(df):
    df['time_poss'] = df['time_poss'].apply(time_to_float)
    return df

# convert record to % win/loss
# treat tie as 0.5 points, wins effectively 1 point
def record_to_float(x):
    splits = list(map(int, x.split('-')))
    total = sum(splits)
    if len(splits) == 3:
        return (splits[0] + 0.5*splits[2]) / total
    else: 
        return splits[0] / total


def team_record_to_float(df):
    df['record'] = df['record'].apply(record_to_float)
    return df

In [85]:
team_stats = pd.read_csv('./final_stats/final_season_team_stats2.csv', index_col=0)
team_stats = team_stats.set_index(['team','week'])
team_stats = team_stats.drop(['wk'], axis=1)
team_stats = team_stats.fillna(0)
team_stats = drop_redundant_cols(team_stats)
team_stats = time_of_poss_to_float(team_stats)
# team_stats = team_record_to_float(team_stats)
team_stats

,,day,date,home/away,opponent,score_team,score_opp,pass_cmp_off,pass_att_off,pass_yds_off,pass_tds_off,...,def_pass_att,def_pass_tds,def_pass_ints,off_pass_cmp,def_sacks,def_sack_yds,tm_penalty,tm_penalty_yds,opp_penalty,opp_penalty_yds
team,week,,,,,,,,,,,,,,,,,,,,,
falcons,1,Sun,September 11,home,saints,26,27,20,33,215,0,...,34,2,0,20,4,35,8,55,8,99
saints,1,Sun,September 11,away,falcons,27,26,23,34,234,2,...,33,0,0,23,0,0,8,99,8,55
panthers,1,Sun,September 11,home,browns,24,26,16,27,207,1,...,34,1,0,16,1,9,8,96,9,71
browns,1,Sun,September 11,away,panthers,26,24,18,34,138,1,...,27,1,1,18,4,28,9,71,8,96
bears,1,Sun,September 11,home,49ers,19,10,8,17,105,2,...,28,0,1,8,2,9,3,24,12,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
giants,19,Sun,January 15,away,vikings,31,24,24,35,289,2,...,40,2,0,24,0,0,5,45,3,12
bengals,19,Sun,January 15,home,ravens,24,17,23,32,183,1,...,29,2,1,23,2,17,7,42,4,36
ravens,19,Sun,January 15,away,bengals,17,24,17,29,209,2,...,32,1,0,17,4,26,4,36,7,42


In [4]:
def get_team_wk(team, wk, df=team_stats):
    return df.xs((team,wk), level=[0,1])

def get_team(team, df=team_stats):
    return df.loc[[team]]

In [153]:
def get_rename_dict(n):
    return {
    'score_team': f'score_team_last_{n}',
    'score_opp': f'score_opp_last_{n}',
    'pass_cmp_off': f'pass_cmp_off_last_{n}',
    'pass_att_off': f'pass_att_off_last_{n}',
    'pass_yds_off': f'pass_yds_off_last_{n}',
    'pass_tds_off': f'pass_tds_off_last_{n}',
    # 'ints_off': f'ints_off_last_{n}', #remove
    'sacks_off': f'sacks_off_last_{n}',
    'sacks_yds_off': f'sacks_yds_off_last_{n}',
    # 'pass_yds_per_att': f'pass_yds_per_att_last_{n}', #remove
    # 'pass_net_yds_per_att': f'pass_net_yds_per_att_last_{n}', #remove
    # 'pass_cmp_prcnt': f'pass_cmp_prcnt_last_{n}', #remove
    'pass_qb_rate': f'pass_qb_rate_last_{n}',
    'rush_att_off': f'rush_att_off_last_{n}',
    'rush_yds_off': f'rush_yds_off_last_{n}',
    # 'rush_yds_per_att': f'rush_yds_per_att_last_{n}', #remove
    'rush_tds_off': f'rush_tds_off_last_{n}',
    # 'fg_made': f'fg_made_last_{n}', #remove
    # 'fg_att': f'fg_att_last_{n}', #remove
    '3d_made': f'3d_made_last_{n}',
    '3d_att': f'3d_att_last_{n}',
    'time_poss': f'time_poss_last_{n}', 
    'record': f'record_last_{n}',
    # '4d_made': f'4d_made_last_{n}', #remove
    # '4d_att': f'4d_att_last_{n}',  #remove
    '1st_down_off': f'1st_down_off_last_{n}',
    'turnover_off': f'turnover_off_last_{n}',
    '1st_down_def': f'1st_down_def_last_{n}',
    'tot_yds_def': f'tot_yds_def_last_{n}',
    'pass_yds_def': f'pass_yds_def_last_{n}',
    'rush_yds_def': f'rush_yds_def_last_{n}',
    'turnover_def': f'turnover_def_last_{n}',
    # 'exp_off': f'exp_off_last_{n}', #remove
    'exp_def': f'exp_def_last_{n}',
    # 'exp_st': f'exp_st_last_{n}', #remove
    # 'spread': f'spread_last_{n}', #remove
    'def_rush_att': f'def_rush_att_last_{n}',
    'def_rush_tds': f'def_rush_tds_last_{n}',
    'off_rush_att': f'off_rush_att_last_{n}',
    'off_rush_tds': f'off_rush_tds_last_{n}',
    'def_pass_cmp': f'def_pass_cmp_last_{n}',
    'def_pass_att': f'def_pass_att_last_{n}',
    'def_pass_tds': f'def_pass_tds_last_{n}',
    'def_pass_ints': f'def_pass_ints_last_{n}',
    'off_pass_cmp': f'off_pass_cmp_last_{n}',
    # 'off_pass_att': f'off_pass_att_last_{n}', #remove
    # 'off_pass_tds': f'off_pass_tds_last_{n}', #remove
    # 'off_pass_ints': f'off_pass_ints_last_{n}', #remove
    'def_sacks': f'def_sacks_last_{n}',
    'def_sack_yds': f'def_sack_yds_last_{n}',
    'tm_penalty': f'tm_penalty_last_{n}',
    'tm_penalty_yds': f'tm_penalty_yds_last_{n}',
    'opp_penalty': f'opp_penalty_last_{n}',
    'opp_penalty_yds': f'opp_penalty_yds_last_{n}'
}

In [122]:
def get_ave_values(df):
    return df.mean(numeric_only=True).to_dict()

def get_weighted_mean(df):
    # wts = get_weights(n)
    cols = df.columns[:-1]
    wt_ave = pd.Series(np.average(df[cols], weights=df['weights'], axis=0), cols)
    return  wt_ave

def get_weights(n):
    rng = list(range(1, n+1))
    total = sum(rng)
    return [x / total for x in rng]


### Get Last N Game Averages
Can be weighted

In [154]:
# figure out how to get select rows based on n game parevious
def get_prev_n_mean(df, wk, n, wt_ave=False):
    # get team name
    team = df.index.get_level_values(0).to_list()[0]
    # get all weeks team played for indices
    wk_idx = df.index.get_level_values(1).to_list()
    
    # get index value for the last week
    if wk not in wk_idx:
        # print(f'no game {wk}')
        return 
    end_idx = wk_idx.index(wk)
    
    # get start wk by subtracting n from end wk index value
    start_wk = wk_idx[end_idx-n]
    # print('start_wk', start_wk,  'end_wk', wk-1)
    # slice last n rows
    df_last_n = df.loc[(team, start_wk):(team, wk-1)]

    # drop non-numeric columns
    df_last_n = df_last_n.drop(['day', 'date', 'home/away','opponent', 'record', 'spread'], axis=1)

    if wt_ave == True:
        # calculate weights    
        wts = get_weights(n)
        # attach wts to df for calculation
        df_last_n['weights'] = wts
        # get weighted average values and put in dataframe
        ave_values_df = pd.DataFrame.from_dict([get_weighted_mean(df_last_n)])
    else: 
        # get average values and put in dataframe
        ave_values_df = pd.DataFrame.from_dict([get_ave_values(df_last_n)])
    
    # set week and team columns and use and indices
    ave_values_df[['team','week']] = [team,wk]
    ave_values_df = ave_values_df.set_index(['team','week'])

    # rename columns - DON'T NEED TO DO THIS 
    # SINCE ONLY KEEPING ONE SET OF LAGS IN EACH DATASET NOW
    
    # rename_dict = get_rename_dict(n)
    # ave_values_df = ave_values_df.rename(columns=rename_dict)

    return ave_values_df

### Get Season Averages
Can be weighted

In [158]:
def get_season_mean(df, wk, wt_ave=False):
    # get team name
    team = df.index.get_level_values(0).to_list()[0]
    # get all weeks team played for indices
    wk_idx = df.index.get_level_values(1).to_list()
    
    # get index value for the last week
    if wk not in wk_idx:
        # print(f'no game {wk}')
        return 
    end_idx = wk_idx.index(wk)
    
    # get start wk by subtracting n from end wk index value
    start_wk = 1
    # print('start_wk', start_wk,  'end_wk', wk-1)
    
    # slice last season rows
    df_last_n = df.loc[(team, 1):(team, wk-1)]

    # drop non-numeric columns
    df_last_n = df_last_n.drop(['day', 'date', 'home/away','opponent', 'record', 'spread'], axis=1)

    if wt_ave == True:
        # calculate weights    
        wts = get_weights(len(df_last_n.axes[0]))
        df_last_n['weights'] = wts

        # get weighted average values and put in dataframe
        ave_values_df = pd.DataFrame.from_dict([get_weighted_mean(df_last_n)])
    else:
    # get average values and put in dataframe
        ave_values_df = pd.DataFrame.from_dict([get_ave_values(df_last_n)])
    
    # set week and team columns and use and indices
    ave_values_df[['team','week']] = [team,wk]
    ave_values_df = ave_values_df.set_index(['team','week'])

    # rename columns - DON'T NEED TO DO THIS 
    # SINCE ONLY KEEPING ONE SET OF LAGS IN EACH DATASET NOW
    
    # rename_dict = get_rename_dict('ssn')
    # ave_values_df = ave_values_df.rename(columns=rename_dict)

    return ave_values_df

In [160]:
bears = get_team('bears', team_stats)
len(get_season_mean(bears, 12, True).columns)

38

In [161]:
len(get_prev_n_mean(bears, 12, 3, True).columns)

38

### 2 things to do
### 1. split up datasets, don't include last game plus last 5 games etc.
### 2. clean out stats that I don't want DONE
### 3. Get Season average DONE

### Get average last 5 game stats



In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Spread gets Inputted Here

In [189]:
# for game in wk 18, get last n game stats (not including 18)
def get_spread(df, wk):
    team = df.index.get_level_values(0).to_list()[0]
    return df.xs((team, wk),)['spread']
    
# STOPPED USING THIS, DON'T WANT TO HAVE BOTH LAST GAME AND LAST n GAMES PUT
# def get_average_last_n_games(team_df, wks, n):
#     values = []
#     for wk in wks:
#         wk_idx = team_df.index.get_level_values(1).to_list()
#         if wk not in wk_idx:
#             print(f'no week {wk}')
            
#         else: 
#             spread = get_spread(team_df, wk)
#             last_1_df = get_prev_n_mean(team_df, wk, 1)
#             last_1_df['spread'] = spread
#             if n == 1:
#                 values.append(last_1_df)
#             elif n != 1:
#                 # get last n games averaged
#                 last_n_df = get_prev_n_mean(team_df, wk, n)
#                 last_n_and_1_df = pd.concat([last_1_df, last_n_df], axis=1)
#                 values.append(last_n_and_1_df)
#     stats_df = pd.concat(values)
#     return stats_df
    
def get_average_last_n_games(team_df, wks, n, wt_ave=False):
    values = []
    for wk in wks:
        wk_idx = team_df.index.get_level_values(1).to_list()
        if wk not in wk_idx:
            # print(f'no week {wk}')
            continue
            
        else: 
            last_n_df = get_prev_n_mean(team_df, wk, n, wt_ave)
            spread = get_spread(team_df, wk)
            last_n_df['spread'] = spread
            values.append(last_n_df)
    stats_df = pd.concat(values)
    return stats_df

def get_average_season_games(team_df, wks, wt_ave: False):
    values = []
    for wk in wks:
        wk_idx = team_df.index.get_level_values(1).to_list()
        if wk not in wk_idx:
            # print(f'no week {wk}')
            continue
            
        else: 
            last_n_df = get_season_mean(team_df, wk, wt_ave)
            spread = get_spread(team_df, wk)
            last_n_df['spread'] = spread
            values.append(last_n_df)
    stats_df = pd.concat(values)
    return stats_df

wk_range = list(range(12,20))

team_stats_df = get_average_season_games(bears, wk_range, 5)

### Not sure what the spread part was doing in the old function???

### Checked spread against online spread and good up to this point

In [190]:
test_teams = ['bears','49ers']
teams = team_stats.index.unique(0).to_list()

def get_all_teams(teams, df, n, wt_ave=False):
    dfs = []
    wk_range = list(range(12,20))
    for team in teams:
        team_df = get_team(team, df)
        if n == 'season' :
            team_df_stats = get_average_season_games(team_df, wk_range, wt_ave)
        else :
            team_df_stats = get_average_last_n_games(team_df, wk_range, n, wt_ave)
        dfs.append(team_df_stats)
    all_df = pd.concat(dfs)
    return all_df

In [191]:
all_team_stats_1 = get_all_teams(teams, team_stats, 1)
all_team_stats_3 = get_all_teams(teams, team_stats, 3)
# all_team_stats_5 = get_all_teams(teams, team_stats, 5)
all_team_stats_7 = get_all_teams(teams, team_stats, 7)
all_team_stats_ssn = get_all_teams(teams, team_stats, 'season')

### WEIGHTED AVERAGES
all_team_stats_3_wt = get_all_teams(teams, team_stats, 3, True)
# all_team_stats_5_wt = get_all_teams(teams, team_stats, 5)
all_team_stats_7_wt = get_all_teams(teams, team_stats, 7, True)
all_team_stats_ssn_wt = get_all_teams(teams, team_stats, 'season', True)

In [192]:
print(all_team_stats_1.shape)
print(all_team_stats_3.shape)
# print(all_team_stats_5.shape)
print(all_team_stats_7.shape)
print(all_team_stats_ssn.shape)
print(all_team_stats_3_wt.shape)
print(all_team_stats_7_wt.shape)
print(all_team_stats_ssn_wt.shape)

(226, 39)
(226, 39)
(226, 39)
(226, 39)
(226, 39)
(226, 39)
(226, 39)


In [193]:
all_team_stats_7.head()

score_team  score_opp  pass_cmp_off  pass_att_off  pass_yds_off  \
team    week                                                                    
falcons 12     22.285714  24.714286     14.142857     21.857143    139.285714   
        13     22.000000  24.428571     14.285714     21.857143    147.142857   
        15     20.285714  25.142857     14.285714     23.285714    152.714286   
        16     20.428571  23.142857     15.000000     25.142857    150.142857   
        17     16.428571  20.714286     15.285714     25.857143    146.285714   

              pass_tds_off  sacks_off  sacks_yds_off  pass_qb_rate  \
team    week                                                         
falcons 12        1.428571   2.714286      17.714286     98.557143   
        13        1.428571   2.142857      13.714286     97.957143   
        15        1.285714   2.000000      13.571429     87.842857   
        16        1.142857   2.142857      12.285714     79.942857   
        17        0.714286   2.142857      11.142857     77.271429   

              rush_att_off  ...  def_pass_tds  def_pass_ints  off_pass_cmp  \
team    week                ...                                              
falcons 12       32.857143  ...      1.285714       0.714286     14.142857   
        13       32.571429  ...      1.428571       0.857143     14.285714   
        15       30.857143  ...      1.285714       0.571429     14.285714   
        16       32.285714  ...      1.285714       0.571429     15.000000   
        17       31.714286  ...      1.285714       0.428571     15.285714   

              def_sacks  def_sack_yds  tm_penalty  tm_penalty_yds  \
team    week                                                        
falcons 12     1.285714      8.857143    4.142857       35.142857   
        13     1.285714      8.857143    3.571429       30.142857   
        15     1.285714      8.857143    3.714286       31.285714   
        16     1.142857      8.142857    3.714286       32.428571   
        17     1.000000      7.000000    3.428571       30.142857   

              opp_penalty  opp_penalty_yds  spread  
team    week                                        
falcons 12       6.714286        50.857143     4.0  
        13       6.428571        48.428571    -1.0  
        15       6.714286        48.428571     4.0  
        16       6.857143        52.571429     6.5  
        17       5.714286        42.857143    -6.0  

[5 rows x 39 columns]

In [194]:
all_team_stats_7_wt.head()

score_team  score_opp  pass_cmp_off  pass_att_off  pass_yds_off  \
team    week                                                                    
falcons 12     22.642857  25.285714     14.607143     22.821429    144.071429   
        13     20.321429  23.857143     14.821429     23.607143    150.500000   
        15     18.821429  22.500000     14.500000     24.142857    153.714286   
        16     18.250000  21.464286     14.178571     24.821429    137.785714   
        17     15.392857  19.928571     15.928571     26.785714    153.250000   

              pass_tds_off  sacks_off  sacks_yds_off  pass_qb_rate  \
team    week                                                         
falcons 12        1.392857   2.357143      15.464286     94.860714   
        13        1.285714   1.928571      13.285714     89.271429   
        15        1.178571   1.642857      11.607143     82.657143   
        16        0.857143   2.142857      10.214286     75.196429   
        17        0.571429   2.107143       8.642857     76.310714   

              rush_att_off  ...  def_pass_tds  def_pass_ints  off_pass_cmp  \
team    week                ...                                              
falcons 12       32.214286  ...      1.071429       0.714286     14.607143   
        13       31.250000  ...      1.250000       0.785714     14.821429   
        15       30.107143  ...      1.142857       0.571429     14.500000   
        16       32.142857  ...      1.571429       0.428571     14.178571   
        17       32.321429  ...      1.500000       0.285714     15.928571   

              def_sacks  def_sack_yds  tm_penalty  tm_penalty_yds  \
team    week                                                        
falcons 12     1.678571     11.250000    3.607143       30.178571   
        13     1.357143      9.035714    3.071429       26.392857   
        15     1.035714      6.821429    3.678571       30.857143   
        16     1.214286      8.857143    3.500000       30.535714   
        17     0.928571      6.821429    3.821429       34.428571   

              opp_penalty  opp_penalty_yds  spread  
team    week                                        
falcons 12       7.178571        57.250000     4.0  
        13       6.500000        51.535714    -1.0  
        15       6.642857        46.928571     4.0  
        16       6.464286        46.321429     6.5  
        17       5.000000        34.678571    -6.0  

[5 rows x 39 columns]

In [195]:
all_team_stats_ssn.head()

score_team  score_opp  pass_cmp_off  pass_att_off  pass_yds_off  \
team    week                                                                    
falcons 12     23.545455  24.909091     14.181818     22.818182    154.454545   
        13     22.666667  24.416667     14.250000     23.000000    155.333333   
        15     22.153846  24.000000     14.153846     23.076923    155.692308   
        16     21.857143  23.785714     14.071429     23.285714    150.928571   
        17     21.000000  23.333333     14.600000     23.933333    155.000000   

              pass_tds_off  sacks_off  sacks_yds_off  pass_qb_rate  \
team    week                                                         
falcons 12        1.181818   2.363636      16.272727     89.272727   
        13        1.166667   2.250000      15.666667     88.183333   
        15        1.153846   2.153846      15.000000     86.900000   
        16        1.071429   2.285714      14.500000     84.835714   
        17        1.000000   2.266667      13.933333     84.806667   

              rush_att_off  ...  def_pass_tds  def_pass_ints  off_pass_cmp  \
team    week                ...                                              
falcons 12       32.818182  ...      1.454545       0.818182     14.181818   
        13       32.500000  ...      1.500000       0.833333     14.250000   
        15       32.153846  ...      1.461538       0.769231     14.153846   
        16       32.642857  ...      1.571429       0.714286     14.071429   
        17       32.666667  ...      1.533333       0.666667     14.600000   

              def_sacks  def_sack_yds  tm_penalty  tm_penalty_yds  \
team    week                                                        
falcons 12     1.545455     11.090909    4.090909       32.727273   
        13     1.416667     10.166667    3.916667       31.666667   
        15     1.307692      9.384615    4.076923       32.923077   
        16     1.357143      9.928571    4.000000       32.714286   
        17     1.266667      9.266667    4.066667       33.733333   

              opp_penalty  opp_penalty_yds  spread  
team    week                                        
falcons 12       6.454545        55.818182     4.0  
        13       6.250000        53.500000    -1.0  
        15       6.307692        51.692308     4.0  
        16       6.285714        51.285714     6.5  
        17       5.933333        48.266667    -6.0  

[5 rows x 39 columns]

In [196]:
all_team_stats_1.to_csv('./final_stats/final_game_stats_last_1.csv')
all_team_stats_3.to_csv('./final_stats/final_game_stats_last_3.csv')
all_team_stats_7.to_csv('./final_stats/final_game_stats_last_7.csv')
all_team_stats_ssn.to_csv('./final_stats/final_game_stats_last_ssn.csv')


all_team_stats_3.to_csv('./final_stats/final_game_stats_last_3_wt.csv')
all_team_stats_7.to_csv('./final_stats/final_game_stats_last_7_wt.csv')
all_team_stats_ssn.to_csv('./final_stats/final_game_stats_last_ssn_wt.csv')


sentiment alreday atached to game vs stats so need to add team game stats   
save all the game stats back into a df

## start from here
working on finalizing final stats with sentiment, function works need to run for each week


In [197]:
# all_team_stats_1 = pd.read_csv('./final_stats/final_game_stats_last_1_2.csv', index_col=['team','week'])
# all_team_stats_3 = pd.read_csv('./final_stats/final_game_stats_last_3_2.csv', index_col=['team','week'])
# all_team_stats_5 = pd.read_csv('./final_stats/final_game_stats_last_5_2.csv', index_col=['team','week'])
# all_team_stats_7 = pd.read_csv('./final_stats/final_game_stats_last_7_2.csv', index_col=['team','week'])
# all_team_stats_10 = pd.read_csv('./final_stats/final_game_stats_last_10_2.csv', index_col=['team','week'])

all_team_stats_1 = pd.read_csv('./final_stats/final_game_stats_last_1.csv', index_col=['team','week'])
all_team_stats_3 = pd.read_csv('./final_stats/final_game_stats_last_3.csv', index_col=['team','week'])
all_team_stats_7 = pd.read_csv('./final_stats/final_game_stats_last_7.csv', index_col=['team','week'])
all_team_stats_ssn = pd.read_csv('./final_stats/final_game_stats_last_ssn.csv', index_col=['team','week'])
all_team_stats_3_wt = pd.read_csv('./final_stats/final_game_stats_last_3_wt.csv', index_col=['team','week'])
all_team_stats_7_wt = pd.read_csv('./final_stats/final_game_stats_last_7_wt.csv', index_col=['team','week'])
all_team_stats_ssn_wt = pd.read_csv('./final_stats/final_game_stats_last_ssn_wt.csv', index_col=['team','week'])

In [198]:
all_team_stats_ssn_wt

score_team  score_opp  pass_cmp_off  pass_att_off  pass_yds_off  \
team    week                                                                    
falcons 12     23.545455  24.909091     14.181818     22.818182    154.454545   
        13     22.666667  24.416667     14.250000     23.000000    155.333333   
        15     22.153846  24.000000     14.153846     23.076923    155.692308   
        16     21.857143  23.785714     14.071429     23.285714    150.928571   
        17     21.000000  23.333333     14.600000     23.933333    155.000000   
...                  ...        ...           ...           ...           ...   
bills   14     27.750000  17.416667     23.833333     37.500000    274.500000   
        15     27.153846  17.000000     23.230769     36.692308    263.384615   
        16     27.500000  17.857143     23.357143     36.928571    265.714286   
        18     28.000000  17.533333     22.800000     36.200000    259.466667   
        19     28.437500  17.875000     22.562500     35.875000    258.062500   

              pass_tds_off  sacks_off  sacks_yds_off  pass_qb_rate  \
team    week                                                         
falcons 12        1.181818   2.363636      16.272727     89.272727   
        13        1.166667   2.250000      15.666667     88.183333   
        15        1.153846   2.153846      15.000000     86.900000   
        16        1.071429   2.285714      14.500000     84.835714   
        17        1.000000   2.266667      13.933333     84.806667   
...                    ...        ...            ...           ...   
bills   14        2.083333   2.083333      10.000000     93.208333   
        15        2.000000   2.153846      10.538462     92.492308   
        16        2.142857   2.142857      10.357143     94.335714   
        18        2.133333   2.066667       9.666667     92.800000   
        19        2.187500   2.062500      10.125000     93.487500   

              rush_att_off  ...  def_pass_tds  def_pass_ints  off_pass_cmp  \
team    week                ...                                              
falcons 12       32.818182  ...      1.454545       0.818182     14.181818   
        13       32.500000  ...      1.500000       0.833333     14.250000   
        15       32.153846  ...      1.461538       0.769231     14.153846   
        16       32.642857  ...      1.571429       0.714286     14.071429   
        17       32.666667  ...      1.533333       0.666667     14.600000   
...                    ...  ...           ...            ...           ...   
bills   14       26.750000  ...      1.250000       1.083333     23.833333   
        15       26.538462  ...      1.153846       1.000000     23.230769   
        16       26.714286  ...      1.214286       0.928571     23.357143   
        18       27.000000  ...      1.200000       0.933333     22.800000   
        19       26.875000  ...      1.312500       1.062500     22.562500   

              def_sacks  def_sack_yds  tm_penalty  tm_penalty_yds  \
team    week                                                        
falcons 12     1.545455     11.090909    4.090909       32.727273   
        13     1.416667     10.166667    3.916667       31.666667   
        15     1.307692      9.384615    4.076923       32.923077   
        16     1.357143      9.928571    4.000000       32.714286   
        17     1.266667      9.266667    4.066667       33.733333   
...                 ...           ...         ...             ...   
bills   14     2.583333     17.583333    5.750000       44.250000   
        15     2.692308     19.000000    5.692308       43.538462   
        16     2.642857     18.857143    5.785714       44.071429   
        18     2.600000     18.600000    6.000000       45.933333   
        19     2.500000     18.000000    5.750000       44.312500   

              opp_penalty  opp_penalty_yds  spread  
team    week                                        
falcons 12       6.454545     

In [180]:
weeks = list(range(12,20,1))
weeks

[12, 13, 14, 15, 16, 17, 18, 19]

In [199]:
wk_df_12 = pd.read_csv('./game_stats/wk_12_sentstats.csv', index_col=[0])
# wk_df_12 = wk_df_12.iloc[3:].drop([6])
# wk_df_13 = pd.read_csv('./game_stats/wk_13_sentstats.csv', index_col=[0])
# wk_df_13
wk_df_12.head()

,Week,Day,Date,home,away,home_score,away_score,away_positive96,away_positive24,away_neutral96,...,away_swing,home_positive96,home_positive24,home_neutral96,home_neutral24,home_negative96,home_negative24,home_sentiment_96h,home_sentiment_24h,home_swing
3,12,Sun,2022-11-27,titans,bengals,16,20,885.0,340.0,1098.0,...,-0.010565,472.0,187.0,865.0,286.0,147.0,56.0,0.219003,0.247637,0.028634
4,12,Sun,2022-11-27,browns,buccaneers,23,17,469.0,186.0,701.0,...,0.021515,878.0,356.0,1315.0,527.0,423.0,195.0,0.173930,0.149351,-0.024579
5,12,Sun,2022-11-27,dolphins,texans,30,15,390.0,129.0,753.0,...,0.056979,2795.0,1019.0,1130.0,365.0,277.0,66.0,0.599238,0.657241,0.058003
7,12,Sun,2022-11-27,commanders,falcons,19,13,239.0,80.0,600.0,...,-0.014771,609.0,262.0,766.0,311.0,185.0,82.0,0.271795,0.274809,0.003014
8,12,Sun,2022-11-27,panthers,broncos,23,10,671.0,197.0,804.0,...,-0.010831,114.0,38.0,238.0,58.0,32.0,9.0,0.213542,0.276190,0.062649


In [200]:

def get_home_dict(columns):
    home_dict = {}
    for col in columns:
        home_dict[col] = f'home_{col}'
    return home_dict

def get_away_dict(columns):
    away_dict = {}
    for col in columns:
        away_dict[col] = f'away_{col}'
    return away_dict

# wk_df_12 = pd.read_csv('./game_stats/wk_12_sentstats.csv', index_col=[0])
# wk_df_12 = wk_df_12.iloc[3:].drop([6])

def get_all_records(wk_df, all_team_stats, n_games):
    all_records = []
    idxs = []

    for row in wk_df.itertuples():
        home_tm = row.home
        away_tm = row.away
        wk = row.Week

        # print(f'Week {wk} {away_tm} @ {home_tm}')
        # get home and away stats from the all team stats df
        home_stats_df = all_team_stats.xs((home_tm,wk), level=[0,1])
        away_stats_df = all_team_stats.xs((away_tm,wk), level=[0,1])

        # create a column rename dictionary from current columns for home and away
        home_col_dict = get_home_dict(home_stats_df.columns)    
        away_col_dict = get_away_dict(away_stats_df.columns)

        # rename columns
        home_stats_df = home_stats_df.rename(columns = home_col_dict)
        away_stats_df = away_stats_df.rename(columns = away_col_dict)
        # convert rows to records dict
        home_records = home_stats_df.to_dict('records')
        away_records = away_stats_df.to_dict('records')

        # combine the dictionary into one dict and append to list
        records = {**home_records[0], **away_records[0]}
        all_records.append(records)
        # get index for row and append to list as well
        idxs.append(row.Index)

    # create dataframe from all records
    all_records_df = pd.DataFrame.from_dict(all_records)
    # set index from list of indexs so they match records
    all_records_df = all_records_df.set_index([pd.Index(idxs)])
    # concat wk df and all records df
    new_wk = pd.concat([wk_df, all_records_df], axis = 1)
    new_wk.to_csv(f'./final_stats/combo_sent_game_2/{n_games}_games/final_sent_w_game_wk_{wk}_games_{n_games}.csv')
    return new_wk



In [201]:
n_game_list = ['1', '3','7','ssn', '3_wt', '7_wt', 'ssn_wt']

all_team_stats_dict = {
    '1': all_team_stats_1,
    '3': all_team_stats_3,
    '7': all_team_stats_7,
    'ssn': all_team_stats_ssn,
    '3_wt': all_team_stats_3_wt,
    '7_wt': all_team_stats_7_wt,
    'ssn_wt': all_team_stats_ssn_wt
}


def get_all_stats(n_game_list, all_team_stats_dict):
    for week in list(range(12,20,1)):
        wk_df = pd.read_csv(f'./game_stats/wk_{str(week)}_sentstats.csv', index_col=[0])
        for n in n_game_list:
            print(n)
            all_team_stats = all_team_stats_dict[n]
            get_all_records(wk_df, all_team_stats, n)

list(range(12,20,1))

[12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
get_all_stats(n_game_list, all_team_stats_dict)

In [203]:
import os
for n in n_game_list:
# for n in ['1']:
    print(n)
    files = os.listdir(f'./final_stats/combo_sent_game_2/{n}_games/')
    combined_df = pd.concat([pd.read_csv(f'./final_stats/combo_sent_game_2/{n}_games/{f}') for f in files])
    combined_df = combined_df.drop(['Unnamed: 0'], axis=1)
    combined_df.to_csv(f'./final_stats/final_season_sent_game_last_{n}.csv')

1
3
7
ssn
3_wt
7_wt
ssn_wt


In [204]:
df = pd.read_csv('./final_stats/final_season_sent_game_last_1.csv', index_col=[0])
df.isnull().values.any()
# for col in df.columns.to_list():
#     print( col)

False